## User Analysis Queries
**1. Rank the user by descending reputation score**\
How reputation is calculated on Stack Exchange:
* +10 Your question is voted up.
* +10 Your answer is voted up.
* +15 Your answer is marked as accepted.
* +2 You accept an answer to your own question.
* (Varies) Your answer is awarded a bounty.
* +10 Your article is voted up (business only).\
(source: https://stackoverflowteams.help/en/articles/4396982-reputation)

**2. Rank the user by the number of upvotes received in answer posts.** \
When you vote up, you are moving that content "up" so it will be seen by more people.
* By default, answers are sorted by number of votes.
* Upvotes on a question give the asker +10 reputation.
* Upvotes on an answer give the answerer +10 reputation.\
(source: https://stackoverflow.com/help/privileges)

**3. Rank users by average answer post scores(sum of answer post scores/number of answer posts made)** 
* Score of a post = number of upvotes a post received - number of downvotes a post received.\
(source: https://meta.stackexchange.com/questions/229255/what-is-the-score-of-a-post)

**4. Best answerer for each topic (eg. under each topic, which user has the highest number of accepted answers)**
* Accepted answer: The system allows users to mark an answer as "accepted", in order to indicate which answer solved their problem or most thoroughly answered their question in the end. (source: https://meta.stackexchange.com/questions/5234/how-does-accepting-an-answer-work)
* topic/tag: A tag is a word or phrase that describes the topic of the question. Tags are a means of connecting experts with questions they will be able to answer by sorting questions into specific, well-defined categories. (source: https://meta.stackexchange.com/help/tagging)

In [21]:
import mysql.connector
import pymongo
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

# Using MySQL Database

In [2]:
### Establish Connection to MySQL
db_config = {
    "host": "127.0.0.1",
    "user": "root",
    "password": "yourpassword", # provide your password to connect to mySQL server
    "database": "stackexchange_cs",
    'auth_plugin': 'mysql_native_password'
}

try:
    # Connect to the database
    connection = mysql.connector.connect(**db_config)

    if connection.is_connected():
        print("Connected to MySQL")

        # Perform database operations here

# if not working, type ALTER USER 'yourusername'@'localhost' IDENTIFIED WITH mysql_native_password BY 'yourpassword' in MySQL workbench;

except Exception as e:
    print(f"Error: {e}")


Connected to MySQL


## SQL Queries

In [3]:
### SQL Queries
# 1. Rank the user by descending reputation score
sql1 = """
    select UserId
        , DisplayName
        , Reputation
        , rank() over(order by Reputation desc) as ReputationRank
    from users 
    where userid <> '-1' # -1 represents stack exchange community
"""


# 2. Rank the user by the number of upvotes received in answer posts.
sql2 = """
SELECT *
	, rank() over(order by a.UpvotesReceived desc) as UpvotesRank
FROM (
	SELECT t1.owneruserid as UserId
		, t3.DisplayName
		, sum(Upvotes_received) as UpvotesReceived
	from ( # posts made by each user
		select postid
			, owneruserid
		from posts
		where posttypeid = 2 # answer post
	) t1
	join ( # number of upvotes each post received
		select postid
			, count(distinct voteid) as Upvotes_received
		from votes
		where votetypeid = '2' # upvote
		group by postid
	) t2
	on t1.postid = t2.postid
	left join users t3
	on t1.owneruserid = t3.userid
	group by t1.owneruserid
) a
;
"""


# 3. Rank users by average answer post scores(sum of answer post scores/number of answer posts made) 
sql3 = """
select a1.UserId
	, a2.DisplayName
	, a1.AvgPostScore
    , rank() over (order by AvgPostScore desc) as ScoreRank
from (
	select owneruserid as UserId
		, count(distinct postid) as AnswerPostCnt
		, sum(score) as TotalAnswerPostScore
		, sum(score)/count(distinct postid) as AvgPostScore
	from posts
    where posttypeid = 2 # answer posts
	group by owneruserid
) a1
left join users a2
on a1.UserId = a2.userid
where a1.AnswerPostCnt > 10;
"""


# 4. Best answerer for each topic (eg. under each topic, return the user who has the highest number of accepted answers)
sql4 = """
select 	final.tagname as Topic
	, final.owneruserid as UserId
    , final.DisplayName
    , AcceptedAnswerCnt
    , AcceptedAnswerRank
from (
	select a1.*
		, a2.DisplayName
		, rank()over(partition by a1.tagname order by a1.AcceptedAnswerCnt desc) as AcceptedAnswerRank 
	from(
		select t3.tagname
			, t2.owneruserid
			, count(distinct t1.answer_id) as AcceptedAnswerCnt
		from ( # question posts and corresponding accepted answers
			select postid
				, acceptedanswerid as answer_id
			from posts
			where posttypeid = 1
		) t1
		join ( # answer post details
			select postid
				, owneruserid
			from posts
			where owneruserid is not null
		) t2
		on t1.answer_id = t2.postid
		join ( # question‘s post tags
			select PostId
				, TagName
			from posttags
		) t3
		on t1.postid = t3.postid
		group by t3.tagname
			, t2.owneruserid
		order by tagname asc, AcceptedAnswerCnt desc
	) a1
    left join users a2
    on a1.owneruserid = a2.userid
) final
where AcceptedAnswerRank = 1
order by tagname asc, DisplayName asc
;
"""

In [4]:
# run sql query and return result as a dataframe
def getSQLQueryResult(sql_query):
    start = time.time()

    cursor = connection.cursor()
    cursor.execute(sql_query)
    result = cursor.fetchall()
    # column names
    columns = [col[0] for col in cursor.description]
    # create result dataframe
    result_df = pd.DataFrame(result, columns = columns)
    # Close the cursor
    cursor.close()

    return result_df, time.time() - start

In [5]:
sql_res1, sql_res1_time = getSQLQueryResult(sql1)
sql_res2, sql_res2_time = getSQLQueryResult(sql2)
sql_res3, sql_res3_time = getSQLQueryResult(sql3)
sql_res4, sql_res4_time = getSQLQueryResult(sql4)
print('All MySQL queries ran successfully')

All MySQL queries ran successfully


In [6]:
# Close SQL Server connection
connection.close()

# Use MongoDB database

In [8]:
### Establish Connection to MongoDB
conn = "mongodb://localhost:27017/"
client = pymongo.MongoClient(conn)
db = client["stackexchange_cs"]

In [9]:
# 1. Rank the user by descending reputation score
c1 = "Users"
p1 = [
    {"$match": {"UserId": {"$ne": -1}}},
    {
        "$project": {
            "UserId": 1,
            "DisplayName": 1,
            "Reputation": 1,
            "_id": 0
        }
    },
    {
        "$setWindowFields": {
            "sortBy": {"Reputation": -1},
            "output": {"ReputationRank": {"$rank": {}}}
        }
    }
]


# 2. Rank the user by the number of upvotes received in answer posts.
c2 = "Posts"
p2 = [
    {
        "$match": {
            "PostTypeId": {"$eq": 2},
            "OwnerUserId": {"$ne": None}
        }
    },
    {"$project": {"PostId": 1, "OwnerUserId": 1}},
    {
        "$lookup": {
            "from": "Votes",
            "localField": "PostId",
            "foreignField": "PostId",
            "as": "VotesReceived"
        }
    },
    {"$unwind": {"path": "$VotesReceived"}},
    {
        "$project": {
            "PostId": 1,
            "OwnerUserId": 1,
            "VoteId": "$VotesReceived.VoteId",
            "VoteTypeId": "$VotesReceived.VoteTypeId"
        }
    },
    {"$match": {"VoteTypeId": {"$eq": 2}}},
    {
        "$group": {
            "_id": "$OwnerUserId",
            "Upvotes": {"$sum": 1}
        }
    },
    {
        "$lookup": {
            "from": "Users",
            "localField": "_id",
            "foreignField": "UserId",
            "as": "UserDetails"
        }
    },
    {"$unwind": {"path": "$UserDetails"}},
    {
        "$project": {
            "_id": 0,
            "UserId": "$_id",
            "DisplayName": "$UserDetails.DisplayName",
            "UpvotesReceived": "$Upvotes"
        }
    },
    {
        "$setWindowFields": {
            "sortBy": {"UpvotesReceived": -1},
            "output": {"UpvotesRank": {"$rank": {}}}
        }
    }
]



# 3. Identify quality of user by average answer post scores(sum of answer post scores/number of answer posts made) 
c3 = "Posts"
p3 = [
    {"$match": {"PostTypeId": {"$eq": 2}}},
    {
        "$group": {
            "_id": "$OwnerUserId",
            "count": {"$sum": 1},
            "totalScore": {"$sum": "$Score"}
        }
    },
    {
        "$project": {
            "_id": 1,
            "totalScore": 1,
            "count": 1,
            "average_score": {"$divide": ["$totalScore", "$count"]}
        }
    },
    {"$match": {"count": {"$gt": 10}}},
    {
        "$lookup": {
            "from": "Users",
            "localField": "_id",
            "foreignField": "UserId",
            "as": "UserDetails"
        }
    },
    {"$unwind": {"path": "$UserDetails"}},
    {
        "$project": {
            "_id": 0,
            "UserId": "$_id",
            "AvgPostScore": "$average_score",
            "DisplayName": "$UserDetails.DisplayName"
        }
    },
    {
        "$setWindowFields": {
            "sortBy": {"AvgPostScore": -1},
            "output": {"ScoreRank": {"$rank": {}}}
        }
    }
]


# 4. Best answerer for each topic (eg. under each topic, return the user who has the highest number of accepted answers)
c4 = 'Posts'
p4 = [
    {"$match": {"PostTypeId": {"$eq": 1}}},
    {
        "$project": {
            "PostId": 1,
            "PostTypeId": 1,
            "AcceptedAnswerId": 1,
            "Body": 1,
            "Title": 1,
            "Tags": 1,
            "_id": 0
        }
    },
    {"$unwind": {"path": "$Tags"}},
    {
        "$lookup": {
            "from": "Posts",
            "localField": "AcceptedAnswerId",
            "foreignField": "PostId",
            "as": "AnswerPostDetail"
        }
    },
    {"$unwind": {"path": "$AnswerPostDetail"}},
    {
        "$group": {
            "_id": {
                "tag": "$Tags",
                "OwnerUserId": "$AnswerPostDetail.OwnerUserId"
            },
            "AcceptedAnswerCnt": {"$count": {}}
        }
    },
    {
        "$project": {
            "_id": 0,
            "Topic": "$_id.tag",
            "UserId": "$_id.OwnerUserId",
            "AcceptedAnswerCnt": 1
        }
    },
    {
        "$setWindowFields": {
            "partitionBy": "$Topic",
            "sortBy": {"AcceptedAnswerCnt": -1},
            "output": {"AcceptedAnswerRank": {"$rank": {}}}
        }
    },
    {"$match": {"AcceptedAnswerRank": {"$eq": 1}}},
    {
        "$lookup": {
            "from": "Users",
            "localField": "UserId",
            "foreignField": "UserId",
            "as": "UserDetail"
        }
    },
    {"$unwind": {"path": "$UserDetail"}},
    {
        "$project": {
            "Topic": 1,
            "UserId": 1,
            "AcceptedAnswerRank": 1,
            "AcceptedAnswerCnt": 1,
            "DisplayName": "$UserDetail.DisplayName"
        }
    },
    {"$sort": {"Topic": 1, "DisplayName":1}}
]

In [10]:
# run mongodb queries
def getMongoResult(collection, pipeline):
    start = time.time()
    result = db[collection].aggregate(pipeline)
    mongo_res = list(result)
    mongo_res = pd.DataFrame(mongo_res)
    return mongo_res, time.time()-start

In [11]:
mongo_res1, mongo_res1_time = getMongoResult(c1, p1)
mongo_res2, mongo_res2_time = getMongoResult(c2, p2)
mongo_res3, mongo_res3_time = getMongoResult(c3, p3)
mongo_res4, mongo_res4_time = getMongoResult(c4, p4)
print('All mongodb queries ran successfully')

All mongodb queries ran successfully


## Result of Query 1 - Rank the user by descending reputation score.

In [12]:
# comparing running time
print(sql_res1_time, mongo_res1_time)

0.2252061367034912 0.7517750263214111


In [56]:
# visualize result of first query from both MySQL and MongoDB
sql_res1 = sql_res1[['UserId', 'DisplayName', 'Reputation', 'ReputationRank']]
mongo_res1 = mongo_res1[['UserId', 'DisplayName', 'Reputation', 'ReputationRank']]
df = pd.concat([sql_res1, mongo_res1], axis = 1)
df.iloc[0:10,:]

,UserId,DisplayName,Reputation,ReputationRank,UserId,DisplayName,Reputation,ReputationRank
0,683,Yuval Filmus,274244,1,683,Yuval Filmus,274244,1
1,755,D.W.,153509,2,755,D.W.,153509,2
2,9550,David Richerby,81298,3,9550,David Richerby,81298,3
3,98,Raphael,71896,4,98,Raphael,71896,4
4,39,Gilles 'SO- stop being evil',43204,5,39,Gilles 'SO- stop being evil',43204,5
5,91753,John L.,38503,6,91753,John L.,38503,6
6,1329,Andrej Bauer,29850,7,1329,Andrej Bauer,29850,7
7,2253,jmite,29609,8,2253,jmite,29609,8
8,4287,Hendrik Jan,29386,9,4287,Hendrik Jan,29386,9
9,17408,gnasher729,27690,10,17408,gnasher729,27690,10


## Result of Query 2 - Rank the user who receive the most upvotes in answer posts.

In [13]:
# comparing running time
print(sql_res2_time, mongo_res2_time)

0.28789710998535156 2.692697048187256


In [57]:
# visualize result of second query from both MySQL and MongoDB
df2 = pd.concat([sql_res2, mongo_res2], axis = 1)
df2.iloc[0:10,:]

,UserId,DisplayName,UpvotesReceived,UpvotesRank,UserId,DisplayName,UpvotesReceived,UpvotesRank
0,683.0,Yuval Filmus,22349,1,683.0,Yuval Filmus,22349.0,1.0
1,755.0,D.W.,11558,2,755.0,D.W.,11558.0,2.0
2,9550.0,David Richerby,7494,3,9550.0,David Richerby,7494.0,3.0
3,98.0,Raphael,5376,4,98.0,Raphael,5376.0,4.0
4,39.0,Gilles 'SO- stop being evil',3553,5,39.0,Gilles 'SO- stop being evil',3553.0,5.0
5,1329.0,Andrej Bauer,2654,6,1329.0,Andrej Bauer,2654.0,6.0
6,91753.0,John L.,2520,7,91753.0,John L.,2520.0,7.0
7,2253.0,jmite,2452,8,2253.0,jmite,2452.0,8.0
8,17408.0,gnasher729,2441,9,17408.0,gnasher729,2441.0,9.0
9,4287.0,Hendrik Jan,2385,10,4287.0,Hendrik Jan,2385.0,10.0


## Result of Query 3 - Rank users by average answer post scores. 

In [14]:
print(sql_res3_time, mongo_res3_time)

0.06373906135559082 0.09431600570678711


In [58]:
# visualize result of third query from both MySQL and MongoDB
mongo_res3 = mongo_res3[['UserId', 'DisplayName','AvgPostScore','ScoreRank']] # reorder column names to match sql result
df3 = pd.concat([sql_res3, mongo_res3], axis = 1)
df3.iloc[0:10,:]

,UserId,DisplayName,AvgPostScore,ScoreRank,UserId,DisplayName,AvgPostScore,ScoreRank
0,72.0,JeffE,16.8776,1,72.0,JeffE,16.877551,1.0
1,169.0,uli,15.2000,2,169.0,uli,15.200000,2.0
2,45.0,Suresh,15.0000,3,45.0,Suresh,15.000000,3.0
3,24.0,Janoma,13.0833,4,24.0,Janoma,13.083333,4.0
4,41.0,Kaveh,12.1126,5,41.0,Kaveh,12.112583,5.0
5,207.0,Uday Reddy,12.0303,6,207.0,Uday Reddy,12.030303,6.0
6,92.0,Alex ten Brink,11.8113,7,92.0,Alex ten Brink,11.811321,7.0
7,568.0,Tsuyoshi Ito,11.6667,8,568.0,Tsuyoshi Ito,11.666667,8.0
8,46.0,sepp2k,11.3333,9,46.0,sepp2k,11.333333,9.0
9,2152.0,Niel de Beaudrap,10.9706,10,2152.0,Niel de Beaudrap,10.970588,10.0


## Comparison of above-mentioned user rank metrics

In [17]:
user_reputation = sql_res1[['UserId','DisplayName','ReputationRank']] 
user_upvotes = sql_res2[['UserId', 'UpvotesRank']]
user_postscore = sql_res3[['UserId', 'ScoreRank']]

In [18]:
user_rank = pd.merge(user_reputation, user_upvotes, on = 'UserId', how = 'inner')
user_rank = pd.merge(user_rank, user_postscore, on = 'UserId', how = 'inner')

In [19]:
user_rank.iloc[0:10,:]

,UserId,DisplayName,ReputationRank,UpvotesRank,ScoreRank
0,683,Yuval Filmus,1,1,174
1,755,D.W.,2,2,229
2,9550,David Richerby,3,3,87
3,98,Raphael,4,4,62
4,39,Gilles 'SO- stop being evil',5,5,28
5,91753,John L.,6,7,257
6,1329,Andrej Bauer,7,6,42
7,2253,jmite,8,8,27
8,4287,Hendrik Jan,9,10,167
9,17408,gnasher729,10,9,368


## Result of Query 4 - Get the best answerer for each topic according to accepted answer count

In [15]:
print(sql_res4_time, mongo_res4_time)

0.17409896850585938 3.6785688400268555


In [98]:
# visualize result of fourth query from both MySQL and MongoDB
mongo_res4 = mongo_res4[['Topic', 'UserId', 'DisplayName', 'AcceptedAnswerCnt', 'AcceptedAnswerRank']]
df4 = pd.concat([sql_res4, mongo_res4], axis = 1)
df4.iloc[0:20,:]

,Topic,UserId,DisplayName,AcceptedAnswerCnt,AcceptedAnswerRank,Topic,UserId,DisplayName,AcceptedAnswerCnt,AcceptedAnswerRank
0,2-sat,683,Yuval Filmus,13,1,2-sat,683.0,Yuval Filmus,13.0,1.0
1,3-sat,683,Yuval Filmus,15,1,3-sat,683.0,Yuval Filmus,15.0,1.0
2,a-star-search,1528,BlueRaja - Danny Pflughoeft,1,1,a-star-search,1528.0,BlueRaja - Danny Pflughoeft,1.0,1.0
3,a-star-search,755,D.W.,1,1,a-star-search,755.0,D.W.,1.0,1.0
4,a-star-search,9550,David Richerby,1,1,a-star-search,9550.0,David Richerby,1.0,1.0
5,a-star-search,91753,John L.,1,1,a-star-search,91753.0,John L.,1.0,1.0
6,a-star-search,151342,LighT,1,1,a-star-search,151342.0,LighT,1.0,1.0
7,a-star-search,97979,Optidad,1,1,a-star-search,97979.0,Optidad,1.0,1.0
8,a-star-search,94615,Pedro Juan Soto,1,1,a-star-search,94615.0,Pedro Juan Soto,1.0,1.0
9,a-star-search,7470,ratchet freak,1,1,a-star-search,89.0,Zach Langley,1.0,1.0


In [31]:
# best answerer for a random selection of topics
rows = np.random.randint(low = 0, high = sql_res4.shape[0], size = 20)
sql_res4.iloc[rows, 0:4]

,Topic,UserId,DisplayName,AcceptedAnswerCnt
665,mu-recursion,98,Raphael,1
650,modelling,755,D.W.,9
440,history,683,Yuval Filmus,4
24,algebra,755,D.W.,4
602,matrix,683,Yuval Filmus,6
315,empirical-research,69,Patrick87,1
413,graphics,755,D.W.,7
487,integer-partitions,1984,j_random_hacker,1
178,computation-models,683,Yuval Filmus,33
619,mergesort,6759,Steven,4
